### main_pm.ipynb – Datenextraktion & Vorverarbeitung

Dieses Notebook umfasst die automatisierte Extraktion und strukturierte Vorverarbeitung der Pressemitteilungen der BVG als Grundlage für die weitere Analyse.

Die zentralen Schritte:
- Automatisiertes Webscraping und lokale Speicherung der HTML-Dateien
- Extraktion von Veröffentlichungsdatum und weiteren Metadaten
- Bereinigung der Inhalte mit BeautifulSoup
- Entfernung von Stoppwörtern und Zählung der Wortfrequenzen pro Mitteilung
- Speicherung der Ergebnisse in einer SQLite-Datenbank

#### 1. Import der benötigten Pakete

In [24]:
# Standard
import os  # Dateipfade und Dateisystem
import time  # Zeitsteuerung
import sys  # Systemfunktionen 

# Datenanalyse
import pandas as pd
import sqlite3  # Speicherung in SQLite-Datenbanken

# Webscraping mit Selenium
import undetected_chromedriver as uc # Browser automatisch steuern
from selenium.webdriver.common.by import By # Elemente auf Webseiten finden
from selenium.webdriver.common.action_chains import ActionChains # Interaktionen auf Webseiten 
from selenium.webdriver.support.ui import WebDriverWait # Warten auf Elemente
from selenium.webdriver.support import expected_conditions as EC # Bedingungen für Webscraping

# Bearbeiten von HTML-Dateien
from bs4 import BeautifulSoup  # HTML auslesen und bereinigen

# Eigene Funktionen (ausgelagert)
sys.path.append("../scripts") # Pfad zu den Funktionen
# Funktionen aus datenaufbereitung.py importieren
from datenaufbereitung import (
    load_stopwords,
    read_html_file,
    process_html
)
stopwords_list = load_stopwords()

In [25]:
# Pfade 

# Input 
SAVE_FOLDER = os.path.join("input", "pm_bvg_raw") # Pfad für html-Dateien

# Output
OUTPUT_FOLDER = os.path.join("output") # Ordner für Output
CSV_PATH = os.path.join("output", "pm_bvg_valid.csv") # Pfad für CSV-Datei
SQL_PATH = os.path.join(OUTPUT_FOLDER, "dwh_pm_bvg.sqlite3") # Pfad für SQL-Datenbank


#### 2. Scraping der BVG-Webseite (Link-Extraktion)

In [26]:
# Scraping starten
print("Starte Webscraping der Pressemitteilungen...")

# Browser öffnen
driver = uc.Chrome()
driver.get("https://www.bvg.de/de/unternehmen/medienportal/pressemitteilungen")
time.sleep(5)

# Cookies akzeptieren 
try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "didomi-notice-agree-button"))
    )
    cookie_button = driver.find_element(By.ID, "didomi-notice-agree-button")

    # Klick per JavaScript 
    driver.execute_script("arguments[0].click();", cookie_button)
    print("[INFO] Cookies akzeptiert.")
    time.sleep(2)

except Exception as e:
    print(f"[ERROR] Cookie konnte nicht geklickt werden: {e}")

all_links = set()  # Leeres Set für Links
current_page = 1   # Start bei Seite 1

while True:
    print(f"[INFO] Seite {current_page}")
    time.sleep(3)

    # URLs sammeln
    links = driver.find_elements(By.CSS_SELECTOR, "a[href^='/de/unternehmen/medienportal/pressemitteilungen/']")
    for link in links:
        href = link.get_attribute("href")
        if href:
            all_links.add(href)

    try:
        # Button für nächste Seite finden und klicken
        next_page_number = str(current_page + 1)
        wait = WebDriverWait(driver, 10)
        next_button = wait.until(EC.presence_of_element_located(
            (By.XPATH, f"//button[normalize-space(text())='{next_page_number}']"))
        )

        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
        time.sleep(1)
        ActionChains(driver).move_to_element(next_button).click().perform()

        current_page += 1
        time.sleep(4)

    except Exception as e:
        print(f"[INFO] Keine weitere Seite gefunden oder Fehler bei Seite {current_page}: {e}")
        break

# Browser schließen
driver.quit()
print(f"[INFO] Anzahl gesammelter Links: {len(all_links)}")

Starte Webscraping der Pressemitteilungen...
[INFO] Cookies akzeptiert.
[INFO] Seite 1
[INFO] Seite 2
[INFO] Seite 3
[INFO] Seite 4
[INFO] Seite 5
[INFO] Seite 6
[INFO] Seite 7
[INFO] Seite 8
[INFO] Seite 9
[INFO] Seite 10
[INFO] Seite 11
[INFO] Seite 12
[INFO] Seite 13
[INFO] Seite 14
[INFO] Seite 15
[INFO] Seite 16
[INFO] Seite 17
[INFO] Seite 18
[INFO] Seite 19
[INFO] Seite 20
[INFO] Seite 21
[INFO] Seite 22
[INFO] Seite 23
[INFO] Seite 24
[INFO] Seite 25
[INFO] Seite 26
[INFO] Seite 27
[INFO] Seite 28
[INFO] Seite 29
[INFO] Seite 30
[INFO] Seite 31
[INFO] Seite 32
[INFO] Seite 33
[INFO] Seite 34
[INFO] Seite 35
[INFO] Seite 36
[INFO] Seite 37
[INFO] Seite 38
[INFO] Seite 39
[INFO] Seite 40
[INFO] Seite 41
[INFO] Seite 42
[INFO] Seite 43
[INFO] Seite 44
[INFO] Seite 45
[INFO] Seite 46
[INFO] Seite 47
[INFO] Seite 48
[INFO] Seite 49
[INFO] Seite 50
[INFO] Seite 51
[INFO] Seite 52
[INFO] Seite 53
[INFO] Seite 54
[INFO] Seite 55
[INFO] Seite 56
[INFO] Seite 57
[INFO] Seite 58
[INFO] Se

In [27]:
# Prüfung
if all_links:
    print(f"\n[INFO] Es wurden {len(all_links)} Pressemitteilungen gefunden.")
    print("[INFO] Beispiel-Links:")
    for link in list(sorted(all_links))[:5]:  # Nur die ersten 5 anzeigen
        print(" -", link)
else:
    print("[WARNUNG] Es wurden keine Links gefunden!")


[INFO] Es wurden 936 Pressemitteilungen gefunden.
[INFO] Beispiel-Links:
 - https://www.bvg.de/de/unternehmen/medienportal/pressemitteilungen/10-uhr-karten-gelten-eine-stunde-frueher
 - https://www.bvg.de/de/unternehmen/medienportal/pressemitteilungen/13-points-go-to
 - https://www.bvg.de/de/unternehmen/medienportal/pressemitteilungen/140-jahre-unter-strom
 - https://www.bvg.de/de/unternehmen/medienportal/pressemitteilungen/155-000-mal-5
 - https://www.bvg.de/de/unternehmen/medienportal/pressemitteilungen/17-000-kostenlose-fahrscheine-s-bahn-berlin-und-bvg-unterstuetzen-ehrenamtliche


#### 3. Download und lokale Speicherung

In [28]:
# Driver starten
driver = uc.Chrome()

# HTML-Inhalt herunterladen
for url in all_links:
    try:
        # Dateinamen aus URL erstellen
        slug = url.strip("/").split("/")[-1]
        filename = f"{slug}.html"
        save_path = os.path.join(SAVE_FOLDER, filename)

        # Wenn schon vorhanden → überspringen
        if os.path.exists(save_path):
            print(f"[INFO] Datei schon vorhanden: {filename}")
            continue

        # Seite aufrufen und HTML holen
        driver.get(url)
        time.sleep(2)  
        content = driver.page_source.encode("utf-8")

        # Speichern
        with open(save_path, "wb") as f:
            f.write(content)
        print(f"[INFO] Gespeichert: {filename}")

    except Exception as e:
        print(f"[FEHLER] bei {url}: {e}")

        # Test: Wenn der Browser weg ist → Neustart
        try:
            _ = driver.title
        except:
            print("[NEUSTART] Browser war nicht mehr erreichbar. Neuer Versuch...")
            try:
                driver.quit()
            except:
                pass  
            driver = uc.Chrome()

# Browser schließen
driver.quit()

[INFO] Datei schon vorhanden: klangvoll-im-untergrund.html
[INFO] Datei schon vorhanden: hasta-la-wista.html
[INFO] Datei schon vorhanden: 2025-01-29-pm-bvg-muva-barrierefrei-ab-maerz-berlinweit.html
[INFO] Datei schon vorhanden: berliner-wasserbetriebe-buddeln-der-tram-den-weg-frei.html
[INFO] Datei schon vorhanden: aussenrum.html
[INFO] Datei schon vorhanden: hintereinanderweg.html
[INFO] Datei schon vorhanden: doppeltes-liftchen.html
[INFO] Datei schon vorhanden: tueren-auf.html
[INFO] Datei schon vorhanden: gemeinsam-fuer-die-mobilitaetswende.html
[INFO] Datei schon vorhanden: dein-anschluss-unter-dieser-nummer.html
[INFO] Datei schon vorhanden: sommer-comeback-fuer-die-u12.html
[INFO] Datei schon vorhanden: zweimal-anders-einmal-neu.html
[INFO] Datei schon vorhanden: sechsstellig-auf-bestellung.html
[INFO] Datei schon vorhanden: mit-bus-und-bahn-zu-den-em-spielen-vbb-bvg-und-s-bahn-ziehen-positive-bilanz.html
[INFO] Datei schon vorhanden: gut-gedaemmt.html
[INFO] Datei schon vorha

In [29]:
# Alle Dateien im Ordner auflisten
html_files = [f for f in os.listdir(SAVE_FOLDER) if f.endswith(".html")]

# Ergebnis anzeigen
print(f"Anzahl gespeicherter html-Dateien: {len(html_files)}")

# Beispielhafte Dateinamen ausgeben
print("Beispieldateien:", html_files[:5])

Anzahl gespeicherter html-Dateien: 936
Beispieldateien: ['10-uhr-karten-gelten-eine-stunde-frueher.html', '13-points-go-to.html', '140-jahre-unter-strom.html', '155-000-mal-5.html', '17-000-kostenlose-fahrscheine-s-bahn-berlin-und-bvg-unterstuetzen-ehrenamtliche.html']


#### 4. Extraktion von Veröffentlichungsdatum und Text

In [30]:
# Liste für die Daten
data = []

# Alle HTML-Dateien durchgehen
files = [f for f in os.listdir(SAVE_FOLDER) if f.endswith(".html")]

for file in files:
    file_path = os.path.join(SAVE_FOLDER, file)
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            html = f.read()

        soup = BeautifulSoup(html, "html.parser") # Links mit BeautifulSoup parsen
        time_tag = soup.find("time") # Time-Element suchen

        if time_tag and time_tag.has_attr("datetime"): # Prüfen, ob Veröffentlichungsdatum vorhanden
            date_str = time_tag["datetime"][:10]  # Nur Datumsformat yyyy-mm-dd 
            year = int(date_str.split("-")[0]) # Jahr extrahieren

            # Pressemitteilungen 2021–2025 übernehmen
            if 2021 <= year <= 2025:
                data.append({
                    "filename": file,
                    "date": date_str,
                    "year": year
                })
            else:
                # Dateien aus anderen Jahren ignorieren
                print(f"Datei {file} aus Jahr {year}, wird ignoriert.")
        else:
            print(f"Kein Datum gefunden in {file}")

    except Exception as e:
        print(f"Fehler bei {file}: {e}")

# DataFrame erstellen
pm_df = pd.DataFrame(data)

Datei 10-uhr-karten-gelten-eine-stunde-frueher.html aus Jahr 2020, wird ignoriert.
Datei 17-000-kostenlose-fahrscheine-s-bahn-berlin-und-bvg-unterstuetzen-ehrenamtliche.html aus Jahr 2018, wird ignoriert.
Datei 21-geteilt-durch-zwei.html aus Jahr 2018, wird ignoriert.
Datei 3-2-1-los.html aus Jahr 2018, wird ignoriert.
Datei 90-baeume-fuer-berlin.html aus Jahr 2019, wird ignoriert.
Datei a1-auf-u3.html aus Jahr 2018, wird ignoriert.
Datei abgeleitet.html aus Jahr 2019, wird ignoriert.
Datei ach-du-jelbe-neune.html aus Jahr 2020, wird ignoriert.
Datei ach-gleis-drauf.html aus Jahr 2020, wird ignoriert.
Datei aehm-2.html aus Jahr 2020, wird ignoriert.
Datei ahoi-ms-tempelhof.html aus Jahr 2018, wird ignoriert.
Datei alexanderpendel-bis-fraesfurter-allee.html aus Jahr 2018, wird ignoriert.
Datei alle-jahre-wieder-2.html aus Jahr 2020, wird ignoriert.
Datei alle-jahre-wieder.html aus Jahr 2018, wird ignoriert.
Datei alle-wege-fuehren-nach-strom.html aus Jahr 2019, wird ignoriert.
Datei all

#### 5. Speicherung der Metadaten (CSV & SQLite)

In [31]:
# Metadaten als CSV-Datei speichern
csv_path = os.path.join(OUTPUT_FOLDER, "pm_bvg_valid.csv")
pm_df.to_csv(csv_path, index=False)
print(f"CSV gespeichert unter {csv_path}")

CSV gespeichert unter output\pm_bvg_valid.csv


In [32]:
# Metadaten als SQL speichern
SQL_PATH = os.path.join(OUTPUT_FOLDER, "dwh_pm_bvg.sqlite3")
conn = sqlite3.connect(SQL_PATH)
pm_df.to_sql("pm_bvg_valid", conn, if_exists="replace", index=False)
conn.close()
print(f"SQL gespeichert unter {SQL_PATH}")

SQL gespeichert unter output\dwh_pm_bvg.sqlite3


#### 6. Definition der Verarbeitungsfunktionen

In [33]:
# Pressemitteilung verarbeiten
def process_press_release(press_release):
    # Nur Dateinamen rausholen
    filename = os.path.basename(press_release["filename"])
    full_path = os.path.join(SAVE_FOLDER, filename)

    # HTML laden mit ausgelagerter Funktion
    html = read_html_file(full_path)

    # Text bereinigen mit ausgelagerter Funktion
    items = process_html(html, stopwords_list)

    # Häufigkeit jedes Wortes zählen
    count = pd.Series(items).value_counts()

    # Als DataFrame formatieren mit Spalten Wort, Medium, Datum
    count_df = count.to_frame()
    count_df.columns = ["count"]
    count_df["word"] = count_df.index
    count_df["source"] = "bvg_pm"  # Quelle festlegen 
    count_df["date"] = press_release["date"]

    return count_df

In [34]:
# Liste für die Ergebnisse-Sammlung 
collection = []

In [35]:
# Funktion: Funktion process_press_release anwenden, in der Variable collection speichern und Erfolg oder Fehler ausgeben
def process_wrapper(press_release):
    filename = press_release["filename"]
    try:
        count = process_press_release(press_release)
        print(f"[INFO] Verarbeitung erfolgreich: {filename}")
        collection.append(count)
    except Exception as e:
        print(f"[ERROR] Fehler bei {filename}: {e}")

#### 7. Anwendung der Verarbeitungsfunktionen auf die Pressemitteilungen

In [36]:
# Verarbeitung aller Pressemitteilungen
pm_df.apply(process_wrapper, axis=1)

# Alle Ergebnisse in einer Tabelle zusammenführen
data = pd.concat(collection, axis=0)
print("Data shape:", data.shape)

[INFO] Verarbeitung erfolgreich: 13-points-go-to.html
[INFO] Verarbeitung erfolgreich: 140-jahre-unter-strom.html
[INFO] Verarbeitung erfolgreich: 155-000-mal-5.html
[INFO] Verarbeitung erfolgreich: 2025-01-03-pm-kleidersammelaktion0.html
[INFO] Verarbeitung erfolgreich: 2025-01-08-pm-zahlen-2024.html
[INFO] Verarbeitung erfolgreich: 2025-01-13-pm-neuer-e-gelenkbus.html
[INFO] Verarbeitung erfolgreich: 2025-01-15-tarif-2025.html
[INFO] Verarbeitung erfolgreich: 2025-01-21-pm-tram-simulatoren.html
[INFO] Verarbeitung erfolgreich: 2025-01-29-pm-bvg-muva-barrierefrei-ab-maerz-berlinweit.html
[INFO] Verarbeitung erfolgreich: 2025-02-04-ideen-die-berlin-bewegen.html
[INFO] Verarbeitung erfolgreich: 2025-02-19-aufsichtsrat-beschliesst-neuaufstellung-des-bvg-vorstands.html
[INFO] Verarbeitung erfolgreich: 2025-03-10-update-stabilitaet-der-bvg0.html
[INFO] Verarbeitung erfolgreich: 2025-03-13-neuer-betriebshof.html
[INFO] Verarbeitung erfolgreich: 2025-03-28-faehr-geht-vor-.html
[INFO] Verarbe

#### 8. Speicherung der Ergebnisse (Wortfrequenzen)

In [37]:
# Wortzählung als CSV speichern
# Zielpfad für CSV definieren
csv_path = os.path.join(OUTPUT_FOLDER, "pm_bvg_wordcount.csv")

# Speichern
data.to_csv(csv_path, index=False)
print(f"[INFO] CSV gespeichert unter: {csv_path}")

[INFO] CSV gespeichert unter: output\pm_bvg_wordcount.csv


In [38]:
# Wortzählung als SQL speichern
# Verbindung zur Datenbank aufbauen 
conn = sqlite3.connect(SQL_PATH)

# Wortzählung speichern 
data.to_sql("wordcount_pm", conn, if_exists="replace", index=False)

# Verbindung schließen
conn.close()
print(f"SQL gespeichert unter {SQL_PATH}")

SQL gespeichert unter output\dwh_pm_bvg.sqlite3


In [39]:
# Prüfung Tabellen in Datenbank
# Verbindung zur Datenbank aufbauen
conn = sqlite3.connect(os.path.join(OUTPUT_FOLDER, "dwh_pm_bvg.sqlite3"))

# Cursor-Objekt erstellen
cursor = conn.cursor()

# Abfrage: Alle Tabellennamen aus der Datenbank holen
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Ergebnis holen
tables = cursor.fetchall()

# Tabellen ausgeben
print("Tabellen in der Datenbank:")
for table in tables:
    print(table[0])

# Verbindung schließen
conn.close()

Tabellen in der Datenbank:
pm_bvg_valid
wordcount_pm
